# Strategy
> Define a family of algorithms, encapsulate each one, and make them interchangeable. Strategy lets the algorithm vary independently from clients that use it.

## Problem
Lets say I have an ML classifier that does a bunch of stuff for training, like forward prop, back prop, optimizing the gradients, etc. After each epoch, it outputs the accuracy of the model. Now if I want to reuse this classifier but have it output precision instead, how do I do it?

In [31]:
import numpy 

class Accuracy:
    def calc(self, y_hat,  y):
        return np.sum(y_hat == y)

class Classifier:
    def __init__(self):
        self.acc_metric = Accuracy()
        
    def fit(self):
        # I am doing a lot of stuff here
        # Forward prop on mini-batches
        # Back prop and optimize
        # After completing one batch
        y_hat = np.array([1, 1, 1, 1], np.float)
        y = np.array([1, 0, 0, 1], np.float)
        acc = self.acc_metric.calc(y_hat, y)
        print(f"Accuracy is {acc:.2f}")

In [32]:
classifer = Classifier()
trainer.fit()

Accuracy is 2.00


## Solution

### OO Solution
Because the metrics calculation is part of a larger task that the classifier does, I don't want to reimplement the classifier for each new type of metric I want to output. In this case the metric calculating strategy can be outsourced to another class. The client calling the classifier decides which metrics calculator to use. And the metric type does not signficantly alter the functionality of the classifier.

In [40]:
from abc import ABC, abstractmethod

class Metric(ABC):
    @abstractmethod
    def calc(self,  y_hat, y):
        pass
    
class Accuracy(Metric):
    def name(self):
        return "Accuracy"
    
    def calc(self, y_hat, y):
        return np.sum(y_hat == y)
    
class Precision(Metric):
    def name(self):
        return "Precision"
    
    def calc(self, y_hat, y):
        return np.sum((y_hat == 1) & (y == 1)) / np.sum(y_hat == 1)
            
class Recall(Metric):
    def name(self):
        return "Recall"
    
    def calc(self, y_hat, y):
        return np.sum((y_hat == 1) & (y == 1)) / np.sum(y == 1)
    
class Classifier:
    def __init__(self, metric):
        self.metric = metric
        
    def fit(self):
        # I am doing a lot of stuff here
        # Forward prop on mini-batches
        # Back prop and optimize
        # After completing one batch
        y = np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])
        y_hat = np.array([0, 0, 0, 1, 1, 1, 1, 1, 1, 1])
        val = self.metric.calc(y_hat, y)
        print(f"{self.metric.name()} is {val:.2f}")

In [41]:
classifier = Classifier(Precision())
classifier.fit()

Precision is 0.71


### Pythonic Solution
Instead of creating a `Metric`s class heirarchy, I can simply pass the appropriate metrics function to the classifier.

In [48]:
def accuracy(y_hat, y):
    return np.sum(y_hat == y)

def precision(y_hat, y):
    return np.sum((y_hat == 1) & (y == 1)) / np.sum(y_hat == 1)

def recall(y_hat, y):
    return np.sum((y_hat == 1) & (y == 1)) / np.sum(y == 1)

class Classifier:
    def __init__(self, metric):
        self.metric = metric
        
    def fit(self):
        # I am doing a lot of stuff here
        # Forward prop on mini-batches
        # Back prop and optimize
        # After completing one batch
        y = np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])
        y_hat = np.array([0, 0, 0, 1, 1, 1, 1, 1, 1, 1])
        val = self.metric(y_hat, y)
        print(f"Generic metric value is {val:.2f}")

In [49]:
classifier = Classifier(recall)
classifier.fit()

Generic metric value is 1.00
